In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import pycuda.driver as drv
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda.autoinit import context, device

file_path = os.getcwd()
attributes = drv.Context.get_device().get_attributes()
sms = attributes[drv.device_attribute.MULTIPROCESSOR_COUNT]
mcr = attributes[drv.device_attribute.MEMORY_CLOCK_RATE]
bus = attributes[drv.device_attribute.GLOBAL_MEMORY_BUS_WIDTH]
l2s = attributes[drv.device_attribute.L2_CACHE_SIZE]

print(f"{device.name()}: {sms} SMs, {mcr*1000*bus*2*1e-9/8:.0f} GBps, {l2s/1024**2:.1f} MB L2\n")

NVIDIA A100-SXM-80GB: 108 SMs, 2039 GBps, 40.0 MB L2



In [4]:
# This is a simple kernel used when the number of elements to be reduced in a row/column is small
# We simply iterate over the elements to be reduced in a loop and accumulate.

def sum_axis_0(Y, X, dim):

    code1 = r"""
#define DIM_M 128
#define X(i,j) X[(i)*lda+(j)]
#define Y(i,j) Y[(i)*lda+(j)]
extern "C"
__global__ void sum_axis_0_k1(float* __restrict__ Y, float* __restrict__ X, const int dim0, const int dim1)
{
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int lda = dim1;
    X = &X(0, bid*DIM_M);
    Y = &Y(0, bid*DIM_M);
    float local_out = 0.0f;
    for (int j=0; j<dim0; j++) {
      local_out += __ldg(&(X(j, tid)));
    }
    Y[tid] = local_out;
}
"""
    # one pass
    sig1    = "PPII"
    DIM_M   = 128
    grid1   = (ceil_div(dim[1], DIM_M), 1, 1)
    block1  = (min(DIM_M, dim[1]), 1, 1)
    params1 = (Y, X, dim[0], dim[1])

    func1 = get_kernel("sum_axis_0_k1", code1, sig1, grid1, block1, params1)

    return func1

def sum_axis_1(Y, X, dim):

    # this might require more than 1 kernel (particually if you're avoiding atomics)
    code1 = r"""
#define DIM_M 128
#define X(i,j) X[(i)*lda+(j)]
extern "C"
__global__ void sum_axis_1_k1(float* __restrict__ Y, float* __restrict__ X, const int dim0, const int dim1)
{
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int lda = dim1;
    X = &X(bid*DIM_M, 0);
    Y = Y + bid*DIM_M;
    float local_out = 0.0f;
    for (int j=0; j<dim1; j++) {
      local_out += __ldg(&X(tid, j));
    }
    Y[tid] = local_out;
}
"""

    # one pass
    sig1    = "PPII"
    DIM_M   = 128
    grid1   = (ceil_div(dim[0], DIM_M), 1, 1)
    block1  = (min(DIM_M, dim[0]), 1, 1)
    params1 = (Y, X, dim[0], dim[1])

    func1 = get_kernel("sum_axis_1_k1", code1, sig1, grid1, block1, params1)

    return func1


# When the number of elements to be reduced per row/column is relatively large (> 16k), do classic parallel reduction performed in multiple columns/rows in parallel (y dim)

def sum_axis_0a(Y, X, dim):
    code1 = r"""
// local reduction within a warp
__inline__ __device__ float warp_reduce_axis_0(float val) {
  for (int offset = warpSize/2; offset > 0; offset /= 2)
    val += __shfl_down_sync(0xffffffff, val, offset);
  return val;
}
// reduction within a block
__inline__ __device__ float block_reduce_axis_0(float val) {
  static __shared__ float shared[32];
  int lane = threadIdx.x % warpSize;
  int wid = threadIdx.x / warpSize;
  val = warp_reduce_axis_0(val);
  if (!lane) shared[wid]=val;
  __syncthreads();
  val = (threadIdx.x < blockDim.x / warpSize) ? shared[lane] : 0.0f;
  if (!wid) val = warp_reduce_axis_0(val);
  return val;
}
// grid reduce
extern "C"
__global__ void sum_axis_0_grid_reduce(float* __restrict__ Y, float* __restrict__ X, const int dim0, const int dim1)
{
  float sum = 0.0f;
  for (int i = blockIdx.x * blockDim.x + threadIdx.x; i<dim1; i += blockDim.x * gridDim.x) {
    sum += __ldg(&X[dim0*i+blockIdx.y]);
  }
  sum = block_reduce_axis_0(sum);
  if (!threadIdx.x) {
    Y[blockIdx.y+dim0*blockIdx.x] = sum;
  }
}
// grid reduce variant which improves memory access pattern slightly
extern "C"
__global__ void sum_axis_0_grid_reduce_4(float* __restrict__ Y, float* __restrict__ X, const int dim0, const int dim1)
{
  float sum[4] = {0.0f, 0.0f, 0.0f, 0.0f};
  for (int i = blockIdx.x * blockDim.x + threadIdx.x; i<dim1; i += blockDim.x * gridDim.x) {
    #pragma unroll
    for (int j=0; j<4; j++) {
      sum[j] += __ldg(&X[dim0*i+(4*blockIdx.y+j)]);
    }
  }
  #pragma unroll 4
  for (int j=0; j<4; j++) {
    sum[j] = block_reduce_axis_0(sum[j]);
  }
  if (!threadIdx.x) {
    #pragma unroll 4
    for (int j=0; j<4; j++) {
      Y[(4*blockIdx.y+j)+dim0*blockIdx.x] = sum[j];
    }
  }
}
"""
    P = drv.mem_alloc(dim[1] * 1024 * 4)

    sig1    = "PPII"
    threads = 256
    blocks = min(ceil_div(dim[0], threads), 1024)
    grid1   = (blocks, dim[1], 1)
    grid2   = (1, dim[1], 1)
    grid1_4   = (blocks, dim[1]//4, 1)
    grid2_4   = (1, dim[1]//4, 1)
    block1  = (threads, 1, 1)
    block2  = (blocks, 1, 1)

    # one pass ok
    if blocks == 1:
      P = Y

    params1 = (P, X, dim[1], dim[0])
    params2 = (Y, P, dim[1], blocks)

    func0a = get_kernel("sum_axis_0_grid_reduce", code1, sig1, grid1, block1, params1)
    func0a4 = get_kernel("sum_axis_0_grid_reduce_4", code1, sig1, grid1_4, block1, params1)
    func0b = get_kernel("sum_axis_0_grid_reduce", code1, sig1, grid2, block2, params2)
    func0b4 = get_kernel("sum_axis_0_grid_reduce_4", code1, sig1, grid2_4, block2, params2)

    def func_0():
      # some heuristic for choosing a slightly faster kernel
      func0a() if dim[1] < 8 else func0a4()

      # 2 passes needed
      if blocks > 1:
        func0b() if dim[1] < 8 else func0b4()

    return func_0

# same algorithm for slighly modified for axis 1

def sum_axis_1a(Y, X, dim):
    code1 = r"""
__inline__ __device__ float warp_reduce_axis_1(float val) {
  for (int offset = warpSize/2; offset > 0; offset /= 2)
    val += __shfl_down_sync(0xffffffff, val, offset);
  return val;
}
__inline__ __device__ float block_reduce_axis_1(float val) {
  static __shared__ float shared[32];
  int lane = threadIdx.x % warpSize;
  int wid = threadIdx.x / warpSize;
  val = warp_reduce_axis_1(val);
  if (!lane) shared[wid]=val;
  __syncthreads();
  val = (threadIdx.x < blockDim.x / warpSize) ? shared[lane] : 0.0f;
  if (!wid) val = warp_reduce_axis_1(val);
  return val;
}
extern "C"
__global__ void sum_axis_1_grid_reduce(float* __restrict__ Y, float* __restrict__ X, const int dim0, const int dim1)
{
  float sum = 0.0f;
  for (int i = blockIdx.x * blockDim.x + threadIdx.x; i<dim1; i += blockDim.x * gridDim.x) {
    sum += __ldg(&X[dim1*blockIdx.y+i]);
  }
  sum = block_reduce_axis_1(sum);
  if (!threadIdx.x) {
    Y[blockIdx.y*gridDim.x+blockIdx.x] = sum;
  }
}
"""
    P = drv.mem_alloc(dim[0] * 1024 * 4)

    sig1    = "PPII"
    threads = 256
    blocks = min(ceil_div(dim[1], threads), 1024)
    grid1   = (blocks, dim[0], 1)
    grid2   = (1, dim[0], 1)
    block1  = (threads, 1, 1)
    block2  = (blocks, 1, 1)
    params1 = (P, X, dim[0], dim[1])
    params2 = (Y, P, dim[0], blocks)

    func1a = get_kernel("sum_axis_1_grid_reduce", code1, sig1, grid1, block1, params1)
    func1b = get_kernel("sum_axis_1_grid_reduce", code1, sig1, grid2, block2, params2)

    def func_1():
      func1a()

      # 2 passes needed
      if blocks > 1:
        func1b()

    return func_1

####################
# Helper code

def get_kernel(name, code, sig, grid, block, params):

    # you might find it easier to develop the cuda code in separate files
    if code[-3:] == ".cu":
        with open(code) as file:
            code = file.read()

    module = SourceModule(code, include_dirs=[file_path], no_extern_c=True)
    kernel = module.get_function(name)
    kernel.prepare(sig)

    def func():
        kernel.prepared_call(grid, block, *params)

    return func

# Potentially handy
def ceil_div(x, y):
    return -(-x // y)

def test_reduce(dim, axis, repeat=1, ones=False, out=False):

    if ones:
        # possibly helpful for debugging
        x = np.ones(dim, dtype=np.float32)
    else:
        x = np.random.uniform(-1.0, 1.0, dim).astype(np.float32)

    y = np.empty(dim[1-axis], dtype=np.float32)

    X = drv.mem_alloc(x.nbytes)
    Y = drv.mem_alloc(y.nbytes)
    drv.memcpy_htod(X, x)

    # some heuristic to select a kernel based on the dimension
    # this needs more tuning
    if axis == 1:
      kernel_func = sum_axis_1a if dim[1] >= 8192 else sum_axis_1
    else:
      kernel_func = sum_axis_0a if dim[0] >= 32768 else sum_axis_0

    gpu_func = kernel_func(Y, X, dim)

    start = drv.Event()
    end   = drv.Event()

    for r in range(repeat):
        gpu_func() # warm up the clock

    start.record()
    for r in range(repeat):
        gpu_func()
    end.record()
    end.synchronize()

    # compute the approximate memory bandwidth
    nbytes = y.nbytes + x.nbytes
    ms     = end.time_since(start) / repeat
    gbps   = nbytes / (ms * 1e6)

    # get result and compare to numpy calculation
    # Compute the numpy sum in float64 for the highest accuracy baseline.
    # There's no need to use float64 inside your kernels, but accuracy is still an important metric.
    drv.memcpy_dtoh(y, Y)
    z = np.sum(x.astype(np.float64), axis=axis).astype(np.float32)
    d = np.abs(y - z)

    l2_err = np.sqrt(np.square(d).sum()) / np.sqrt(np.square(z).sum())

    print("GBps:%4.0f ms:%7.3f dim: (%8d,%8d) axis:%d max_err: %8.3f max_val: %8.3f l2_err: %7.5f" % (
        gbps, ms, dim[0], dim[1], axis, d.max(), y.max(), l2_err ))

    if out:
        # inspect your output to debug
        fmt = "%5.0f" if ones else "%5.2f"
        #np.savetxt("out_dif.txt", d, fmt=fmt)
        ##np.savetxt("out_cpu.txt", z, fmt=fmt)
        #np.savetxt("out_gpu.txt", y, fmt=fmt)
        print(ms, nbytes, gbps, x.shape, z.shape)
        exit()

####################
# Execution code

ones   = 0
out    = 0
repeat = 10 # use a large repeat count to measure bandwidth more accurately

# feel free to play with this to see how robust your performance is to different shapes
for exp0 in range(26):
    exp1 = 26 - exp0 - 1
    dim0 = 2 ** exp0
    dim1 = 2 ** exp1

    # reduce axis=1
    test_reduce((dim0, dim1), axis=1, repeat=repeat, ones=ones, out=out)
    # reduce axis=0
    test_reduce((dim0, dim1), axis=0, repeat=repeat, ones=ones, out=out)

GBps: 636 ms:  0.211 dim: (       1,33554432) axis:1 max_err:    0.001 max_val: 3475.572 l2_err: 0.00000
GBps: 816 ms:  0.329 dim: (       1,33554432) axis:0 max_err:    0.000 max_val:    1.000 l2_err: 0.00000
GBps: 792 ms:  0.169 dim: (       2,16777216) axis:1 max_err:    0.000 max_val: -218.910 l2_err: 0.00000
GBps:1050 ms:  0.192 dim: (       2,16777216) axis:0 max_err:    0.000 max_val:    1.999 l2_err: 0.00000
GBps: 876 ms:  0.153 dim: (       4, 8388608) axis:1 max_err:    0.000 max_val: 1303.801 l2_err: 0.00000
GBps:1538 ms:  0.109 dim: (       4, 8388608) axis:0 max_err:    0.000 max_val:    3.972 l2_err: 0.00000
GBps: 853 ms:  0.157 dim: (       8, 4194304) axis:1 max_err:    0.000 max_val: 1806.895 l2_err: 0.00000
GBps:1494 ms:  0.101 dim: (       8, 4194304) axis:0 max_err:    0.000 max_val:    6.999 l2_err: 0.00000
GBps: 801 ms:  0.168 dim: (      16, 2097152) axis:1 max_err:    0.000 max_val: 1523.933 l2_err: 0.00000
GBps:1486 ms:  0.096 dim: (      16, 2097152) axis:0 ma